In [ ]:
import torch
import cutlass
import cutlass.cute as cute
from cutlass.cute.runtime import from_dlpack
import cute_viz

In [ ]:
@cute.jit
def fhw_viz(f: cutlass.Constexpr[int], h: cutlass.Constexpr[int], w: cutlass.Constexpr[int]):
    S = cute.make_layout(((f, h, w),(f, h, w)))
    # print(S.stride[::-1])
    # S = cute.make_layout(((f, h, w),(f, h, w)), stride=S.stride[::-1])
    cute_viz.display_layout(S)
    print(S)

fhw_viz(2, 3, 5)

In [ ]:
# s = torch.load("shmil_layout_viz/attention_matrix_S_block1_counter12.pt")
s = torch.load("../attention_matrix_S_block1_counter12.pt")
p = torch.softmax(s, dim=-1)
p_sorted = p.sort(dim=1).values

In [ ]:
import torch.nn.functional as F
max_res = 512
attn_down = F.adaptive_max_pool2d(
# attn_down = F.adaptive_avg_pool2d(
    p_sorted[None],  # (1, 1, seq_len_q, seq_len_k)
    output_size=(max_res, max_res)
)[0]  # -> (1, 1, max_res, max_res)
attn_down.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(6, 6))
attn_cpu = attn_down.detach().float().cpu()
plt.imshow(attn_cpu, cmap='viridis', interpolation='nearest')
# plt.title(f"Batch {b} | Head {h} | Percentage {perecentage * 100:.2f}% | Do Softmax: {do_softmax}")
plt.axis("off")
plt.tight_layout()